# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are:

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [37]:
from collections import defaultdict, Counter

def load_fivegrams(fivegram_path):
    fivegram = []
    with open(fivegram_path, "r", encoding="latin-1") as file:
        for line in file:
            parts = line.strip().split("\t")
            if len(parts) == 6:
                count = int(parts[0])
                w1, w2, w3, w4, w5 = parts[1], parts[2], parts[3], parts[4], parts[5]
                fivegram.append((count, w1, w2, w3, w4, w5))
    return fivegram

fivegram = load_fivegrams("fivegrams (2).txt")
fivegram_dict = {(w1, w2, w3, w4, w5): count for count, w1, w2, w3, w4, w5 in fivegram_counts}

fourgram = defaultdict(int)
for count, w1, w2, w3, w4, _ in fivegram:
    fourgram[(w1, w2, w3, w4)] += count

fivegram_probs = {(w1, w2, w3, w4, w5): count / fourgram[(w1, w2, w3, w4)]
                  for (w1, w2, w3, w4, w5), count in fivegram_dict.items()}

In [38]:
keyboard = {
    'q': (0, 0), 'w': (1, 0), 'e': (2, 0), 'r': (3, 0), 't': (4, 0),
    'y': (5, 0), 'u': (6, 0), 'i': (7, 0), 'o': (8, 0), 'p': (9, 0),
    'a': (0, 1), 's': (1, 1), 'd': (2, 1), 'f': (3, 1), 'g': (4, 1),
    'h': (5, 1), 'j': (6, 1), 'k': (7, 1), 'l': (8, 1),
    'z': (0, 2), 'x': (1, 2), 'c': (2, 2), 'v': (3, 2), 'b': (4, 2),
    'n': (5, 2), 'm': (6, 2)
}

In [39]:
import math

def calculate_dist(c1, c2):
    x1, y1 = keyboard.get(c1, (0, 0))
    x2, y2 = keyboard.get(c2, (0, 0))
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

#from https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
def weighted_levenshtein(s1, s2):
    len_s1 = len(s1)
    len_s2 = len(s2)
    dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]
    for i in range(len_s1 + 1):
        dp[i][0] = i
    for j in range(len_s2 + 1):
        dp[0][j] = j
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = calculate_dist(s1[i - 1], s2[j - 1]) / 2
            dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)
    return dp[len_s1][len_s2]

In [40]:
def generate_candidates(word, vocabulary, max_dist=2):
    candidates = []
    for candidate in vocabulary:
        if weighted_levenshtein(word, candidate) <= max_dist:
            candidates.append(candidate)
    return candidates

In [41]:
def get_best(context, word, vocabulary, fivegram_probs):
    candidates = generate_candidates(word, vocabulary)
    best_candidate = None
    best_score = -1
    for candidate in candidates:
        fivegram_prob = fivegram_probs.get((context[0], context[1], context[2], context[3], candidate), 1e-8)
        keyboard_distance = calculate_keyboard_distance(word[-1], candidate[-1])
        score = fivegram_prob / (keyboard_distance + 1e-8)
        if score > best_score:
            best_score = score
            best_candidate = candidate
    return best_candidate

In [42]:
def correct_sentence(sentence, vocabulary, fivegram_probs):
    words = sentence.split()
    corrected_sentence = []
    for i in range(len(words)):
        if i >= 4:
            context = (words[i-4], words[i-3], words[i-2], words[i-1])
            if words[i] not in vocabulary:
                candidate = get_best(context, words[i], vocabulary, fivegram_probs)
                corrected_sentence.append(candidate)
            else:
                corrected_sentence.append(words[i])
        else:
            corrected_sentence.append(words[i])
    return " ".join(corrected_sentence)

In [43]:
vocabulary = {w for fivegram in fivegram_dict.keys() for w in fivegram}

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I chose Fivegrams because they capture more context than bigrams or trigrams, allowing them to better correct errors in long sentences or complex cases.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)